# LLM Query via Vertex AI Studio (Gemini)
A minimal agent: build a prompt → call Gemini → quick self-check → save artifacts to GCS.

In [ ]:
import os, json, uuid
from google.cloud import storage, aiplatform
from vertexai.generative_models import GenerativeModel

PROJECT_ID = os.getenv('PROJECT_ID') or input('Project ID: ')
REGION = os.getenv('REGION', 'us-central1')
BUCKET = os.getenv('BUCKET') or input('Bucket (no gs://): ')
RUN_ID = uuid.uuid4().hex[:8]

aiplatform.init(project=PROJECT_ID, location=REGION)
model = GenerativeModel('gemini-1.5-pro')
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET)

In [ ]:
def write_gcs(path, obj):
    blob = bucket.blob(path)
    if isinstance(obj, (dict, list)):
        obj = json.dumps(obj, indent=2)
    blob.upload_from_string(str(obj), content_type='application/json')

goal = "Outline a quickstart to deploy a Flask API on Cloud Run."
prompt = f"You are a helpful agent. Provide steps and a checklist. Goal: {goal}"
resp = model.generate_content(prompt)
answer = resp.text.strip()

check = {"ok_len": len(answer.split()) < 160, "has_step": "step" in answer.lower()}
artifacts = {"goal": goal, "answer": answer, "check": check}

write_gcs(f'llm_query/{RUN_ID}/result.json', artifacts)
if check['ok_len'] and check['has_step']:
    write_gcs('llm_query/latest/success.json', {"status":"SUCCESS","run_id":RUN_ID})
artifacts